In [ ]:

import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/goemotions/README.md
/kaggle/input/goemotions/goemotions_model_card.pdf
/kaggle/input/goemotions/GoEmotionsFormat.PNG
/kaggle/input/goemotions/replace_emotions.py
/kaggle/input/goemotions/extract_words.py
/kaggle/input/goemotions/calculate_metrics.py
/kaggle/input/goemotions/analyze_data.py
/kaggle/input/goemotions/tables/emotion_words.csv
/kaggle/input/goemotions/data/test.tsv
/kaggle/input/goemotions/data/ekman_mapping.json
/kaggle/input/goemotions/data/dev.tsv
/kaggle/input/goemotions/data/emotions.txt
/kaggle/input/goemotions/data/train.tsv
/kaggle/input/goemotions/data/ekman_labels.csv
/kaggle/input/goemotions/data/sentiment_dict.json
/kaggle/input/goemotions/data/sentiment_mapping.json
/kaggle/input/goemotions/data/full_dataset/goemotions_2.csv
/kaggle/input/goemotions/data/full_dataset/goemotions_1.csv
/kaggle/input/goemotions/data/full_dataset/goemotions_3.csv
/kaggle/input/goemotions/plots/hierarchical_corr.pdf
/kaggle/input/goemotions/plots/number_of_labels.pdf


In [2]:
!pip install transformers==4.30.2 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.18.0 requires transformers>=4.33.1, but you have transformers 4.30.2 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification


In [4]:
DATA_DIR = "/kaggle/input/goemotions/data"

train_df = pd.read_csv(f"{DATA_DIR}/train.tsv", sep="\t", header=None, names=["text","labels","id"])
dev_df   = pd.read_csv(f"{DATA_DIR}/dev.tsv", sep="\t", header=None, names=["text","labels","id"])
test_df  = pd.read_csv(f"{DATA_DIR}/test.tsv", sep="\t", header=None, names=["text","labels","id"])


In [5]:
NUM_LABELS = 28

def to_multihot(label_str):
    indices = list(map(int, label_str.split(",")))
    arr = np.zeros(NUM_LABELS)
    arr[indices] = 1
    return arr

train_df["multihot"] = train_df["labels"].apply(to_multihot)
dev_df["multihot"]   = dev_df["labels"].apply(to_multihot)
test_df["multihot"]  = test_df["labels"].apply(to_multihot)


In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def encode(batch):
    return tokenizer(
        batch["text"].tolist(),
        truncation=True,
        padding="max_length",
        max_length=128
    )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
train_enc = encode(train_df)
dev_enc   = encode(dev_df)
test_enc  = encode(test_df)


In [8]:
train_dataset = TensorDataset(
    torch.tensor(train_enc["input_ids"]),
    torch.tensor(train_enc["attention_mask"]),
    torch.tensor(np.vstack(train_df["multihot"].values), dtype=torch.float)
)

dev_dataset = TensorDataset(
    torch.tensor(dev_enc["input_ids"]),
    torch.tensor(dev_enc["attention_mask"]),
    torch.tensor(np.vstack(dev_df["multihot"].values), dtype=torch.float)
)

test_dataset = TensorDataset(
    torch.tensor(test_enc["input_ids"]),
    torch.tensor(test_enc["attention_mask"]),
    torch.tensor(np.vstack(test_df["multihot"].values), dtype=torch.float)
)


In [9]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader   = DataLoader(dev_dataset, batch_size=batch_size)


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

model.to(device)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [11]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

loss_fn = torch.nn.BCEWithLogitsLoss()


In [12]:
from tqdm import tqdm

epochs = 2

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids, attention_masks, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Training Loss = {total_loss/len(train_loader)}")


Training Epoch 1: 100%|██████████| 2714/2714 [08:20<00:00,  5.42it/s]


Epoch 1 Training Loss = 0.12049474904976659


Training Epoch 2: 100%|██████████| 2714/2714 [08:42<00:00,  5.20it/s]

Epoch 2 Training Loss = 0.08253311691565512


In [13]:
model.eval()
val_loss = 0

with torch.no_grad():
    for batch in tqdm(dev_loader, desc="Validation"):
        input_ids, attention_masks, labels = [b.to(device) for b in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_masks,
            labels=labels
        )

        val_loss += outputs.loss.item()

print("Validation Loss:", val_loss / len(dev_loader))


Validation: 100%|██████████| 340/340 [00:23<00:00, 14.26it/s]

Validation Loss: 0.08417984497021226


In [14]:
SAVE_PATH = "/kaggle/working/emotion_model"

model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print("Model Saved →", SAVE_PATH)


Model Saved → /kaggle/working/emotion_model
